In [1]:
from sklearn import datasets
cancer = datasets.load_breast_cancer()

In [2]:
import pandas as pd

cancer_df = pd.DataFrame(cancer['data'], columns=cancer['feature_names'])
cancer_df['type'] = cancer['target']
cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,type
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [3]:
import torch

class CancerDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        cols = [col for col in dataframe.columns if col != 'target']
        self.parameters = torch.from_numpy(dataframe[cols].values).type(torch.float32)
        self.targets = torch.from_numpy(dataframe['type'].values).type(torch.float32)
        self.targets = self.targets.reshape((len(self.targets), 1))

    def __len__(self):
        return len(self.parameters)

    def __getitem__(self, idx):
        parameters = self.parameters[idx]
        target = self.targets[idx]
        return parameters, target

ds = CancerDataset(cancer_df)

In [4]:
train_ds, valid_ds = torch.utils.data.random_split(ds, [int(0.8*len(ds)), len(ds) - int(0.8*len(ds))], generator=torch.Generator().manual_seed(42))
len(train_ds), len(valid_ds)

(455, 114)

In [5]:
from torch.utils.data import DataLoader

BS_train = 256
BS_val = 128

train_dl = DataLoader(train_ds, batch_size=BS_train, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=BS_val, shuffle=False)

In [6]:
from torch import nn

class CancerNeuralNetwork(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_layers=[100, 50, 20]):
        super().__init__()
        self.network = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, hidden_layers[0]),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layers[0], hidden_layers[1]),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layers[1], hidden_layers[2]),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layers[2], num_outputs),
        )
        self.activation = torch.nn.Sigmoid()

    def forward(self, x):
        logits = self.network(x)
        probs = self.activation(logits)
        return logits, probs

In [7]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [8]:
import subprocess as sp
import os

def get_gpu_memory():
    command = "nvidia-smi --query-gpu=memory.total --format=csv"
    memory_total_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
    memory_total_values = [int(x.split()[0]) for i, x in enumerate(memory_total_info)]

    command = "nvidia-smi --query-gpu=memory.free --format=csv"
    memory_free_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
    memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]

    command = "nvidia-smi --query-gpu=memory.used --format=csv"
    memory_used_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
    memory_used_values = [int(x.split()[0]) for i, x in enumerate(memory_used_info)]
    return memory_total_values, memory_free_values, memory_used_values

total, free, used = get_gpu_memory()
print(f"GPU memory: total: {total} MiB, free: {free} MiB, used: {used} MiB")

GPU memory: total: [4096] MiB, free: [3708] MiB, used: [195] MiB


In [9]:
model = CancerNeuralNetwork(31, 1)
model.to(device)

CancerNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=31, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=20, bias=True)
    (5): ReLU()
    (6): Linear(in_features=20, out_features=1, bias=True)
  )
  (activation): Sigmoid()
)

In [10]:
total, free, used = get_gpu_memory()
print(f"GPU memory: total: {total} MiB, free: {free} MiB, used: {used} MiB")

GPU memory: total: [4096] MiB, free: [2891] MiB, used: [1012] MiB


In [11]:
LR = 1e-2

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

In [43]:
num_prints = 4

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # X and y to device
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        logits, probs = model(X)
        loss = loss_fn(logits, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % (int(len(dataloader)/num_prints)+1) == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [44]:
total, free, used = get_gpu_memory()
print(f"GPU memory: total: {total} MiB, free: {free} MiB, used: {used} MiB")

GPU memory: total: [4096] MiB, free: [2677] MiB, used: [1226] MiB


In [45]:
0%4

0

In [46]:
epochs = 14
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dl, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
0 2 4 0.5 0
loss: 0.757278  [    0/  455]
1 2 4 0.5 0
loss: 0.683460  [  199/  455]
Epoch 2
-------------------------------
0 2 4 0.5 0
loss: 0.652309  [    0/  455]
1 2 4 0.5 0
loss: 0.662991  [  199/  455]
Epoch 3
-------------------------------
0 2 4 0.5 0
loss: 0.636595  [    0/  455]
1 2 4 0.5 0
loss: 0.654617  [  199/  455]
Epoch 4
-------------------------------
0 2 4 0.5 0
loss: 0.620117  [    0/  455]
1 2 4 0.5 0
loss: 0.665559  [  199/  455]
Epoch 5
-------------------------------
0 2 4 0.5 0
loss: 0.634096  [    0/  455]
1 2 4 0.5 0
loss: 0.640139  [  199/  455]
Epoch 6
-------------------------------
0 2 4 0.5 0
loss: 0.622272  [    0/  455]
1 2 4 0.5 0
loss: 0.610223  [  199/  455]
Epoch 7
-------------------------------
0 2 4 0.5 0
loss: 0.679013  [    0/  455]
1 2 4 0.5 0
loss: 0.660700  [  199/  455]
Epoch 8
-------------------------------
0 2 4 0.5 0
loss: 0.560853  [    0/  455]
1 2 4 0.5 0
loss: 0.558254  [  199/  455]
Epoch 9


In [47]:
total, free, used = get_gpu_memory()
print(f"GPU memory: total: {total} MiB, free: {free} MiB, used: {used} MiB")

GPU memory: total: [4096] MiB, free: [2677] MiB, used: [1226] MiB


In [67]:
command = "nvidia-smi --query-gpu=memory.total --format=csv"
command = "nvidia-smi"
memory_total_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
memory_total_info = sp.check_output(command.split()).decode('ascii').split('\n')
processes = memory_total_info[14]
memory_total_info, processes, type(memory_total_info), len(memory_total_info)

(['Fri Jun 24 00:20:04 2022       ',
  '+-----------------------------------------------------------------------------+',
  '| NVIDIA-SMI 515.43.04    Driver Version: 515.43.04    CUDA Version: 11.7     |',
  '|-------------------------------+----------------------+----------------------+',
  '| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |',
  '| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |',
  '|                               |                      |               MIG M. |',
  '|===============================+======================+======================|',
  '|   0  Quadro T1000        On   | 00000000:01:00.0  On |                  N/A |',
  '| N/A   48C    P8     7W /  N/A |   1226MiB /  4096MiB |      0%      Default |',
  '|                               |                      |                  N/A |',
  '+-------------------------------+----------------------+----------------------+',
  '                         